In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import chi2, SelectKBest, RFE
from sklearn.metrics import accuracy_score, classification_report

In [6]:
# Load Data
df = pd.read_csv("Titanic-Dataset.csv")

# Separate Survived
y = df["Survived"].astype(int)
X = df.drop(columns=["Survived"])

# Create a list of numeric and categorical variables
# Will be used for the feature selection process
# Names will not be used as they are irrelevant
num_cols = ["Age", "SibSp", "Parch", "Fare"]
cat_cols = ["Pclass", "Sex", "Embarked"]

In [7]:
# Feature Extraction with Pipeline

# Numeric Data
num_tf = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

# Categorical Data
cat_tf = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

In [8]:
# Create FamilySize Feature
X["FamilySize"] = X["SibSp"].fillna(0) + X["Parch"].fillna(0) + 1

# Add FamilySize to numeric group
preprocess = ColumnTransformer([
    ("num", num_tf, num_cols + ["FamilySize"]),
    ("cat", cat_tf, cat_cols),
])

In [11]:
# Feature selection with SelectKBest
# That function will use variance analysis
# Read: https://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection

from sklearn.feature_selection import f_classif
selector_filter = SelectKBest(score_func=f_classif, k=5)

# Create final pipeline (THIS IS ONLY PIPELINE, NOT PROCESSING DATA YET)
pipe_filter = Pipeline([
    ("prep", preprocess), # run pipeline preprocessing
    ("sel", selector_filter), # run pipeline feature selection
    ("clf", LogisticRegression(max_iter=1000)) # test using simple model -> Logistic Regression
])

In [12]:
# Training and testing the model
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

pipe_filter.fit(X_train, y_train)
pred = pipe_filter.predict(X_test)
print("=== Filter (ANOVA) + LR ===")
print("Accuracy:", accuracy_score(y_test, pred))
print(classification_report(y_test, pred))

=== Filter (ANOVA) + LR ===
Accuracy: 0.776536312849162
              precision    recall  f1-score   support

           0       0.80      0.85      0.82       110
           1       0.73      0.67      0.70        69

    accuracy                           0.78       179
   macro avg       0.77      0.76      0.76       179
weighted avg       0.77      0.78      0.77       179



In [13]:
# 1) Nama fitur setelah preprocess
feat_names = pipe_filter.named_steps["prep"].get_feature_names_out()
print("Nama fitur:", feat_names)
print("\n")

# 2) Mask & skor fitur terpilih (SelectKBest)
sel = pipe_filter.named_steps["sel"]
mask = sel.get_support()
selected_names = feat_names[mask]
selected_scores = sel.scores_[mask]
top = sorted(zip(selected_names, selected_scores), key=lambda t: t[1], reverse=True)[:10]
print("Top fitur:", top)

Nama fitur: ['num__Age' 'num__SibSp' 'num__Parch' 'num__Fare' 'num__FamilySize'
 'cat__Pclass_1' 'cat__Pclass_2' 'cat__Pclass_3' 'cat__Sex_female'
 'cat__Sex_male' 'cat__Embarked_C' 'cat__Embarked_Q' 'cat__Embarked_S']


Top fitur: [('cat__Sex_female', 306.5932488951883), ('cat__Sex_male', 306.59324889518797), ('cat__Pclass_3', 80.33862734392042), ('cat__Pclass_1', 73.99727564291717), ('num__Fare', 58.31490728198491)]
